In [13]:
import pandas as pd
from openai import OpenAI

In [8]:
data=pd.read_csv("data/pubchem_dump_with_wiki_text.csv")

In [9]:
data.columns

Index(['Unnamed: 0', 'text', 'mentioned_entity', 'cid', 'name', 'properties',
       'wiki_text', 'combined_text'],
      dtype='object')

In [11]:
multi_hop=[]
row=data[data.mentioned_entity!='[]'].iloc[0]
mentioned_entity=[int(item)-1 for item in eval(row['mentioned_entity']) if int(item)<=50000]
for edge in mentioned_entity:
    next_row=data.iloc[edge]
    if len(next_row['combined_text'])>10:
        qa={'source':row['name'],'bridge':next_row['name'],'text1':row['combined_text'],'text2':next_row['combined_text'],}
        multi_hop.append(qa)

In [12]:
multi_hop

[{'source': 'Acetyl-DL-carnitine',
  'bridge': 'Acetic Acid',
  'text1': 'wikipedia: \n pubchem:\nO-acetylcarnitine is an O-acylcarnitine having acetyl as the acyl substituent. It has a role as a human metabolite. It is functionally related to an acetic acid. It is a conjugate base of an O-acetylcarnitinium.\nAcetylcarnitine is an investigational drug in the United states, Italy, United Kingdom, China, Israel, and Norway, and it is approved in Italy, Portugal, Argentina, Chile, Philippines, Australia, and India. Acetylcarnitine can be synthesized, but it is also naturally found in adequate amounts in healthy humans. In human plasma and tissues, acetylcarnitine is the most predominant acylated ester of carnitine, which is an amino acid derivative that is made in the kidney, liver, and brain from lysine and methionine. The main role of acetylcarnitine is to help transport fatty acids into the mitochondrial matrix where fatty acid metabolism occurs.\nAcetylcarnitine has been reported in D

{'source': 'Acetyl-DL-carnitine',
 'bridge': 'Acetic Acid',
 'text1': 'wikipedia: \n pubchem:\nO-acetylcarnitine is an O-acylcarnitine having acetyl as the acyl substituent. It has a role as a human metabolite. It is functionally related to an acetic acid. It is a conjugate base of an O-acetylcarnitinium.\nAcetylcarnitine is an investigational drug in the United states, Italy, United Kingdom, China, Israel, and Norway, and it is approved in Italy, Portugal, Argentina, Chile, Philippines, Australia, and India. Acetylcarnitine can be synthesized, but it is also naturally found in adequate amounts in healthy humans. In human plasma and tissues, acetylcarnitine is the most predominant acylated ester of carnitine, which is an amino acid derivative that is made in the kidney, liver, and brain from lysine and methionine. The main role of acetylcarnitine is to help transport fatty acids into the mitochondrial matrix where fatty acid metabolism occurs.\nAcetylcarnitine has been reported in Dros

In [20]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given two texts and a bridge word to connect them.\n\n"
    f"Text1: {multi_hop[0]['text1']}\n"
    f"Bridge Word: {multi_hop[0]['bridge']}\n"
    f"Text2: {multi_hop[0]['text2']}\n\n"
    f"Your job is to generate a multihop question that requires the user to read the first document find the bridge word and using the bridge word go to the second document. "
    f"You should not mention the bridge word directly so it requires the person to read the first document first"
    f"the question should be a single one with a closed brief answer"
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
print (response.choices[0].message.content.strip())

{ 
  "q": "In which countries is the investigational drug, synthesized from an amino acid derivative that is made in the kidney, liver, and brain from lysine and methionine, approved for use, and what notable role does its carboxylic acid precursor play in household and food industries?", 
  "a": "The investigational drug is approved in Italy, Portugal, Argentina, Chile, Philippines, Australia, and India. Its carboxylic acid precursor plays an important role as a descaling agent in households and as an acidity regulator in the food industry." 
}


In [22]:
print(multi_hop[0]['text1'])

wikipedia: 
 pubchem:
O-acetylcarnitine is an O-acylcarnitine having acetyl as the acyl substituent. It has a role as a human metabolite. It is functionally related to an acetic acid. It is a conjugate base of an O-acetylcarnitinium.
Acetylcarnitine is an investigational drug in the United states, Italy, United Kingdom, China, Israel, and Norway, and it is approved in Italy, Portugal, Argentina, Chile, Philippines, Australia, and India. Acetylcarnitine can be synthesized, but it is also naturally found in adequate amounts in healthy humans. In human plasma and tissues, acetylcarnitine is the most predominant acylated ester of carnitine, which is an amino acid derivative that is made in the kidney, liver, and brain from lysine and methionine. The main role of acetylcarnitine is to help transport fatty acids into the mitochondrial matrix where fatty acid metabolism occurs.
Acetylcarnitine has been reported in Drosophila melanogaster, Schizosaccharomyces pombe, and other organisms with da

In [40]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given a text and a bridge word.\n\n"
    f"Text: {multi_hop[0]['text1']}\n"
    f"Bridge Word: {multi_hop[0]['bridge']}\n"
    f"Your job is to generate a question that requires the user to read the  document and find the bridge word as the answer. "
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
q1= eval(response.choices[0].message.content.strip())

In [41]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given a text and a bridge word.\n\n"
    f"Text: {multi_hop[0]['text2']}\n"
    f"Key Word: {multi_hop[0]['bridge']}\n"
    f"Your job is to generate a question about the Key word "
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
q2= eval(response.choices[0].message.content.strip())

In [42]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given two question with their answers and a bridge word.\n\n"
    f"question1: {q1['q']}\n"
    f"answer1: {q1['a']}\n"
    f"question2: {q2['q']}\n"
    f"answer2: {q2['a']}\n"
    f"Your job is to generate a multi-hop question, that combines these two questions.  "
    f"The question should require the person to find the answer to the first question then look for the answer of the second question"
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content.strip())

{"q": "What is the primary industrial method for the production of the compound that O-acetylcarnitine is functionally related to, and what are the main catalysts used in this process?", "a": "The primary industrial method for the production of acetic acid, which O-acetylcarnitine is functionally related to, is the carbonylation of methanol. The main catalysts used in this process are rhodium-based catalysts for the Monsanto process and iridium-based catalysts for the Cativa process."}


In [38]:
prompt

"You are given two question with their answers and a bridge word.\n\nquestion1: What compound is O-acetylcarnitine functionally related to?\nanswer1: Acetic Acid\nquestion2: What is the primary industrial method used to produce acetic acid worldwide?\nanswer2: The primary industrial method used to produce acetic acid worldwide is the carbonylation of methanol, specifically through the Cativa process, which uses an iridium-catalyzed reaction to produce acetic acid more efficiently.\nYour job is to generate a multi-hop question, that combines these two questions.  The question should require the person to find the answer to the first question then look for the answer of the second questionReturn a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "